### Comparison of the Optimus pipeline results from wlg (V6.0.0) and broad institute (V5.8.4)
#### Aim of this part of analysis is identify the strength of the differences between the two version of Optimus pipelines and to understand the reasons for the differences

#### 1. Load necessary libraries and useful functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from basicAtlas import terra
import dalmatian as dm
import collections
import numpy as np

In [3]:
import fsspec
import anndata
import pandas as pd
from typing import Union

def read_file_from_url(url: str) -> Union[anndata.AnnData, pd.DataFrame]:
    """Read a file from a URL and return the appropriate data structure.

    Parameters
    ----------
    url : str
        The URL of the file to read.

    Returns
    -------
    output : Union[anndata.AnnData, pd.DataFrame]
        The data read from the file, which can be either an AnnData object (if ".h5ad" in url)
        or a Pandas DataFrame (if ".csv" in url).
    """

    with fsspec.open(url) as f:
        if ".h5ad" in url:
            output = anndata.read_h5ad(f)
        elif "Summary.csv" in url:
            output = pd.read_csv(f, header=None) #Read the CSV file with the first row as data
        elif ".csv" in url:
            output = pd.read_csv(f)
        else:
            raise ValueError("Unsupported file format. Supported formats are .h5ad and .csv.")
    
    return output

In [34]:
def compare_2_df(df1: pd.DataFrame, df2: pd.DataFrame) -> None:
    """Compare two Pandas DataFrames and print the differences.

    Parameters
    ----------
    df1 : pd.DataFrame
        The first DataFrame to compare.
    df2 : pd.DataFrame
        The second DataFrame to compare.

    Returns
    -------
    None
    """

    # Identify common columns
    common_columns = df1.columns.intersection(df2.columns)

    # Select common columns from each DataFrame
    common_df1 = df1[common_columns]
    common_df2 = df2[common_columns]

    # Compare the selected subsets and store the differences
    differences = common_df1.compare(common_df2)
    print(differences)

    # Check if there are differences
    if differences.empty:
        print("The common parts of the DataFrames are identical.")
    else:
        print("Differences in the common parts of the DataFrames:")
        print(differences)

#### 2. Data uploading to WLG bucket

In [4]:
# current directory
current_directory = os.getcwd()
print(current_directory)

/Users/xiliu/Documents/analysis/terraPipelines/notebook


In [8]:
# Upload the primary annotation to WLG bucket
#!gsutil cp ../scAtlas/tmp/gencode.v27.primary_assembly.annotation.gtf "gs://whitelabgx-references/hg38/gencode.v27.primary_assembly.annotation.gtf"

Copying file://../scAtlas/tmp/gencode.v27.primary_assembly.annotation.gtf [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][  1.1 GiB/  1.1 GiB]   31.9 MiB/s                                   
Operation completed over 1 objects/1.1 GiB.                                      


#### 3. Comparison of the STARsolo summary

In [4]:
# WLG's summary file
wlg_star_url = "gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/submissions/07e6d89b-3db8-436e-bc50-fef6b40cdd04/Optimus/934d7faa-21e2-452d-a624-ea8896142b81/call-STARsoloFastq/shard-1/cacheCopy/Summary.csv"
wlg_star_summary = read_file_from_url(wlg_star_url)

In [27]:
# HCA's summary file
hca_star_url ="gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/submissions/ca3a5a8d-3a78-4a10-8ea6-4f4f8eb26c5d/Optimus/600fabcf-7af4-45e4-9ada-c759ed1d6d5f/call-STARsoloFastq/shard-1/Summary.csv"
hca_star_summary = read_file_from_url(hca_star_url)

In [30]:
display(wlg_star_summary)
display(hca_star_summary)

,0,1
0,Number of Reads,319930655
1,Reads With Valid Barcodes,0.973214
2,Sequencing Saturation,0.667937
3,Q30 Bases in CB+UMI,0.931203
4,Q30 Bases in RNA read,0.901736
5,Reads Mapped to Genome: Unique+Multiple,0.961733
6,Reads Mapped to Genome: Unique,0.889793
7,Reads Mapped to Gene: Unique+Multiple Gene,NoMulti
8,Reads Mapped to Gene: Unique Gene,0.47964
9,Estimated Number of Cells,5356


,0,1
0,Number of Reads,319930655
1,Reads With Valid Barcodes,0.973214
2,Sequencing Saturation,0.667937
3,Q30 Bases in CB+UMI,0.931203
4,Q30 Bases in RNA read,0.901736
5,Reads Mapped to Genome: Unique+Multiple,0.961733
6,Reads Mapped to Genome: Unique,0.889793
7,Reads Mapped to Gene: Unique+Multiple Gene,NoMulti
8,Reads Mapped to Gene: Unique Gene,0.47964
9,Estimated Number of Cells,5356


In [31]:
# Compare the two summary files
differences = hca_star_summary.ne(wlg_star_summary)
differences

,0,1
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


#### 4. Load and compare the results (h5ad) from the two pipelines

##### 4.1. Fetch results using broad institute HCA_Optimus_Pipeline: pbmc human 10k v3 S1

In [5]:
hca_h5ad_url = "gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/submissions/ca3a5a8d-3a78-4a10-8ea6-4f4f8eb26c5d/Optimus/600fabcf-7af4-45e4-9ada-c759ed1d6d5f/call-OptimusH5adGeneration/pbmc_human_v3.h5ad"
hca_adata = read_file_from_url(hca_h5ad_url)

/opt/homebrew/Caskroom/miniconda/base/envs/bulk/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
hca_adata
#display(hca_adata.obs)
#display(hca_adata.var.tail(10))

AnnData object with n_obs × n_vars = 1136912 × 58347
    obs: 'cell_names', 'CellID', 'emptydrops_Limited', 'emptydrops_IsCell', 'n_reads', 'noise_reads', 'perfect_molecule_barcodes', 'reads_mapped_uniquely', 'reads_mapped_multiple', 'spliced_reads', 'antisense_reads', 'n_molecules', 'n_fragments', 'fragments_with_single_read_evidence', 'molecules_with_single_read_evidence', 'perfect_cell_barcodes', 'reads_mapped_too_many_loci', 'n_genes', 'genes_detected_multiple_observations', 'emptydrops_Total', 'molecule_barcode_fraction_bases_above_30_mean', 'molecule_barcode_fraction_bases_above_30_variance', 'genomic_reads_fraction_bases_quality_above_30_mean', 'genomic_reads_fraction_bases_quality_above_30_variance', 'genomic_read_quality_mean', 'genomic_read_quality_variance', 'reads_per_fragment', 'fragments_per_molecule', 'cell_barcode_fraction_bases_above_30_mean', 'cell_barcode_fraction_bases_above_30_variance', 'n_mitochondrial_genes', 'n_mitochondrial_molecules', 'pct_mitochondrial_molec

In [32]:
# Find duplicate variable (gene) names
hca_var_names = hca_adata.var_names
hca_duplicates = [item for item, count in collections.Counter(hca_var_names).items() if count > 1]

print("Duplicate variable names:", hca_duplicates)
print("Number of duplicate variable:", len(hca_duplicates))

Duplicate variable names: ['U6', 'TP73-AS1', 'Y_RNA', 'SNORA77', 'SCARNA16', 'SNORA70', 'SCARNA11', 'SCARNA17', 'SCARNA18', 'snoU13', 'SNORA44', 'SNORA16A', 'SCARNA24', 'Metazoa_SRP', 'uc_338', 'SNORA62', 'SNORA63', 'SNORD46', 'SNORD38B', 'SNORA26', 'SNORA58', 'DLEU2_6', 'DLEU2_5', 'DLEU2_4', 'DLEU2_3', 'DLEU2_2', 'DLEU2_1', 'SNORA31', 'SNORA2', 'SNORD81', 'SNORA51', 'SNORA25', 'SNORA42', 'U3', 'SNORA40', '7SK', 'U1', 'U2', '5S_rRNA', 'U6atac', 'U4', 'SNORD59', 'SCARNA4', 'SNORD64', 'ACA64', 'RGS5', 'SCARNA20', 'U7', 'SNORA67', 'SNORA72', 'SNORD60', 'SNORD116', 'U8', 'LINC01115', 'SNORD18', 'SCARNA21', 'SNORA36', 'SNORD75', 'TMEM247', 'STPG4', 'SNORA75', 'SNORA12', 'SNORD78', 'ACA59', 'SNORA74', 'snoU109', 'SNORA19', 'ACTR3BP2', 'DAOA-AS1_2', 'SCARNA15', 'SNORA48', 'SNORD56', 'PDE11A', 'SNORA43', 'SNORA17', 'PCGEM1', 'SNORA4', 'SNORD70', 'SNORD11', 'SNORA1', 'Vault', 'SNORD51', 'SCARNA6', 'SNORD39', 'LINC01238', 'GHRLOS', 'SNORD5', 'SNORA64', 'SNORD77', 'PRSS50', 'CYB561D2', 'SNORD19B'

In [ ]:
# Subset the AnnData var to only include the duplicated variables
for target_var_name in hca_duplicates:
    print("Variable name:", target_var_name)
    print("Duplication quantity:", hca_adata.var[hca_adata.var_names == target_var_name].shape[0])
    print("Variable:", hca_adata.var[hca_adata.var_names == target_var_name])

In [33]:
# Variable names are not unique. To make them unique
hca_adata.var_names_make_unique()

/opt/homebrew/Caskroom/miniconda/base/envs/bulk/lib/python3.9/site-packages/anndata/utils.py:111: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(


##### 4.2. Fetch results from WLG pipeline

In [8]:
wm = dm.WorkspaceManager(TERRA_WS)
output_df = wm.get_sample_sets()
display(output_df)
#display(output_df.columns)

#wlg_h5ad_url = output_df.loc['optimus_V6.0.0_wlg', 'h5ad_output_file']
#wlg_adata = read_file_from_url(wlg_h5ad_url)

,aligner_metrics,bam,cell_barcodes_csv,cell_calls,cell_metrics,gene_metrics,genomic_reference_version,h5ad_output_file,html_report_array,matrix,matrix_col_index,matrix_row_index,metrics_csv_array,samples,summary_pdf
sample_set_id,,,,,,,,,,,,,,,
optimus_V6.0.0_wlg,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,[gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...,[gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/...,"[pbmc_10k_v3_S1_L001, pbmc_10k_v3_S1_L002]",gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/s...


##### For this part of analysis, we use te code of chunk below to read the data, 
##### because the results of this job used the same parameters as the broad institute pipeline

In [6]:
wlg_h5ad_url = "gs://fc-447aee29-8362-4c0b-b8d0-b3b10eb9e2a6/submissions/07e6d89b-3db8-436e-bc50-fef6b40cdd04/Optimus/934d7faa-21e2-452d-a624-ea8896142b81/call-OptimusH5adGeneration/10k_pbmc_v3.h5ad"
wlg_adata = read_file_from_url(wlg_h5ad_url)

/opt/homebrew/Caskroom/miniconda/base/envs/bulk/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [9]:
wlg_adata
#display(wlg_adata.obs)
#display(wlg_adata.var)

AnnData object with n_obs × n_vars = 1136912 × 58347
    obs: 'cell_names', 'CellID', 'emptydrops_Limited', 'emptydrops_IsCell', 'n_reads', 'noise_reads', 'perfect_molecule_barcodes', 'reads_mapped_exonic', 'reads_mapped_exonic_as', 'reads_mapped_intronic', 'reads_mapped_intronic_as', 'reads_mapped_uniquely', 'reads_mapped_multiple', 'duplicate_reads', 'spliced_reads', 'antisense_reads', 'n_molecules', 'n_fragments', 'fragments_with_single_read_evidence', 'molecules_with_single_read_evidence', 'perfect_cell_barcodes', 'reads_mapped_intergenic', 'reads_unmapped', 'reads_mapped_too_many_loci', 'n_genes', 'genes_detected_multiple_observations', 'emptydrops_Total', 'molecule_barcode_fraction_bases_above_30_mean', 'molecule_barcode_fraction_bases_above_30_variance', 'genomic_reads_fraction_bases_quality_above_30_mean', 'genomic_reads_fraction_bases_quality_above_30_variance', 'genomic_read_quality_mean', 'genomic_read_quality_variance', 'reads_per_molecule', 'reads_per_fragment', 'fragments

##### 4.3. comparison of h5ad results from the two pipelines

##### 4.3.1. comparison of selection features from the two pipelines

In [36]:
# Compare some important columns in the obs
wlg_obs_df = wlg_adata.obs[['n_reads',
               'perfect_molecule_barcodes',
               'perfect_cell_barcodes',
               'molecule_barcode_fraction_bases_above_30_mean',
               'emptydrops_FDR',
               'emptydrops_Total']]

hca_obs_df = hca_adata.obs[['n_reads',
               'perfect_molecule_barcodes',
               'perfect_cell_barcodes',
               'molecule_barcode_fraction_bases_above_30_mean',
               'emptydrops_FDR',
               'emptydrops_Total']]

compare_2_df(wlg_obs_df, hca_obs_df) 

Empty DataFrame
Columns: []
Index: []
The common parts of the DataFrames are identical.


In [37]:
# Compare some important columns in the var
wlg_var_df = wlg_adata.var[['n_reads',
                'perfect_molecule_barcodes',
                'molecule_barcode_fraction_bases_above_30_mean',
                'reads_per_molecule',
                'molecule_barcode_fraction_bases_above_30_variance']]

hca_var_df = hca_adata.var[['n_reads',
                'perfect_molecule_barcodes',
                'molecule_barcode_fraction_bases_above_30_mean',
                'reads_per_molecule',
                'molecule_barcode_fraction_bases_above_30_variance']]

compare_2_df(wlg_var_df, hca_var_df) 

Empty DataFrame
Columns: []
Index: []
The common parts of the DataFrames are identical.


##### Optional: comparison of adata.X from the two pipelines

###### doesn't work in local machine, because of the memory limitation

In [10]:
# Compare the .X matrices element-wise
x_equal = np.array_equal(wlg_adata.X, hca_adata.X)

if x_equal:
    print("The .X matrices are equal.")
else:
    print("The .X matrices are not equal.")


<__array_function__ internals>:200: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.


: 

##### Optional: comparison of sparse matrix from the two pipelines

###### doesn't work in local machine, because of the memory limitation

In [35]:
# sparse matrix is named 'sparse_matrix'
wlg_sparse_matrix = wlg_adata.X

# Convert the sparse matrix to a dense NumPy array
wlg_dense_matrix = wlg_sparse_matrix.toarray()

# use regular NumPy indexing to see the head
#head = wlg_dense_matrix[:5, :]  # Assuming you want to see the first 5 rows

# Split the first 300000 rows
split_wlg_dense_matrix = wlg_dense_matrix[:300000, :]

: 

In [ ]:
hca_sparse_matrix = hca_adata.X
split_hca_dense_matrix = hca_sparse_matrix.toarray()[:300000, :]

##### draft

In [ ]:
# Find elements in hca_duplicates that are not in wlg_duplicates
difference1 = [item for item in hca_duplicates if item not in wlg_duplicates]

# Find elements in wlg_duplicates that are not in hca_duplicates
difference2 = [item for item in wlg_duplicates if item not in hca_duplicates]

# Print the differences
print("Elements in hca_duplicates that are not in wlg_duplicates:", difference1)
print("Elements in wlg_duplicates that are not in hca_duplicates:", difference2)